<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small><hr style="margin:0;background-color:silver">

# **[🏆💎Diamonds](https://www.kaggle.com/competitions/23jan23cml-diamonds)**

See [**instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running and naming your Colab notebooks.

**(Optioal) CONSENT.** If ok with sharing your Colab for educational purposes, please check the box below with "X".

<mark>[ . . ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purpose. We understand that sharing is optional and this decision will not affect our grade in any way. 

In [1]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [2]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 23jan23cml-diamonds      # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: 23jan23cml-diamonds
Using competition: 23jan23cml-diamonds
 teamId  teamName                      submissionDate       score       
-------  ----------------------------  -------------------  ----------  
9791778  Unet - лучше для мужчины нет  2023-02-04 13:27:40  591.84951   
9808520  6-на мне камни сто карат      2023-02-03 19:07:54  592.66383   
9829303  💎 1                           2023-02-05 20:26:42  595.31923   
9807796  чепухи                        2023-02-04 19:59:55  615.33789   
9820059  (-_-)                         2023-02-02 19:52:37  711.02709   
9809181  decate                        2023-02-05 19:57:42  811.57111   
9791551  💎Baseline 🐍                   2022-01-25 03:19:41  1066.56542  


In [3]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
ToCSV = lambda df, fname: df.to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 8)
tDIR, sDIR = 'trainXY/', 'testY/'

CPU times: user 1.24 s, sys: 182 ms, total: 1.42 s
Wall time: 1.97 s


In [4]:
# df = pd.read_csv('XY_diamonds.csv'); df

In [5]:
# vX = df.query('price!=price').drop('price', axis=1)  # slice a test sample
# tXY = df.query('price==price')                       # slice training sample
# tX, tY = tXY.drop('price', axis=1), tXY.price        # split into training I/O

In [6]:
# def ScatterCorrHist(df):
#   def corrdot(*args, **kwargs):
#     # credit: https://stackoverflow.com/questions/48139899
#     corr_r = args[0].corr(args[1], 'pearson')
#     corr_text = f"{corr_r:2.2f}".replace("0.", ".")
#     ax = plt.gca();
#     ax.set_axis_off();
#     msz = abs(corr_r) * 5000   # marker size
#     fsz = abs(corr_r) * 40 + 5 # font size
#     ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
#     ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

#   sns.set(style='white', font_scale=.8);
#   g = sns.PairGrid(df, aspect=1, diag_sharey=False);
#   g.fig.set_size_inches(20,10)
#   g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
#   g.map_diag(sns.histplot, kde_kws={'color':'black'});
#   g.map_upper(corrdot);
#   g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
#   _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
#   _ = plt.show();

# # ScatterCorrHist(tXY.head(200))

In [7]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
1. Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Perhaps something in EDA or prior experience lead you to these)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

In [8]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [9]:
df_copy = pd.read_csv('/content/XY_diamonds.csv')
testing = df_copy[df_copy['price'] != df_copy['price']]
testing = testing.drop('price', axis=1)
df = df_copy.dropna()

В ходе нашей работы мы попробовали различные способы работы с категориальными признаками, такие как: OneHotEncoding, LabelEncoding и TargetEncoding. LabelEncoding бесспорно показал себя лучше всего, поэтому на нём мы и остановились.

In [10]:
enc1 = LabelEncoder()
enc2 = LabelEncoder()
enc3 = LabelEncoder()

enc1.fit(df['cut'])
enc2.fit(df['color'])
enc3.fit(df['clarity'])

df['cut'] = enc1.transform(df['cut'])
df['color'] = enc2.transform(df['color'])
df['clarity'] = enc3.transform(df['clarity'])
testing['cut'] = enc1.transform(testing['cut'])
testing['color'] = enc2.transform(testing['color'])
testing['clarity'] = enc3.transform(testing['clarity'])

LabelEncoder()

LabelEncoder()

LabelEncoder()

<ipython-input-10-04186982ce2c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cut'] = enc1.transform(df['cut'])
<ipython-input-10-04186982ce2c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['color'] = enc2.transform(df['color'])
<ipython-input-10-04186982ce2c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Здесь мы выводим корреляции между фичами. Так можно было заметить сильную корреляцию у одних признаков и слабую корреляцию у других. Однако в виду малого количества фичей мы не удалили никакие признаки.

In [11]:
# plt.figure(figsize=(24,20), dpi= 80)
# sns.heatmap(df.corr(), xticklabels=df.corr().columns, yticklabels=df.corr().columns, cmap='RdYlGn', center=0, annot=True)

# # Decorations
# plt.title('Correlogram', fontsize=22)
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# plt.show()

Выведем описание нашего DataFrame

In [12]:
# df.describe()

Как мы могли заметить в переменных x,y,z встречаются нули, что однозначно является выбросом, т.к. такого не может быть в реальной жизни.

In [13]:
df = df.drop(df[df['x']==0].index)
df = df.drop(df[df['y']==0].index)
df = df.drop(df[df['z']==0].index)

In [14]:
# df.describe()

Выведем попарные графики переменных, чтобы очистить датасет от выбросов.

In [15]:
# sns.pairplot(df[['carat', 'depth', 'table', 'x','y','z','price']], diag_kind='kde')

In [16]:
df = df.drop(df[(df['z'] < 2) & (df['price'] >= 5000)].index)
df = df.drop(df[df['z'] > 8].index)
df = df.drop(df[df['y'] > 20].index)
df = df.drop(df[df['table'] > 90].index)
df = df.drop(df[df['table'] < 37].index)
df = df.drop(df[df['depth'] < 50].index)
df = df.drop(df[df['depth'] > 85].index)
df = df.drop(df[df['carat'] > 4].index)

In [17]:
# sns.pairplot(df[['carat', 'depth', 'table', 'x','y','z','price']], diag_kind='kde')

In [18]:
df = df.drop(df[df['carat'] > 3.4].index)
df = df.drop(df[df['depth'] > 80].index)
df = df.drop(df[df['depth'] < 57].index)
df = df.drop(df[df['table'] > 80].index)
df = df.drop(df[df['z'] > 6.2].index)
df = df.drop(df[df['x'] > 11].index)
df = df.drop(df[(df['y'] < 4.8) & (df['price'] >= 5000)].index)
df = df.drop(df[df['y'] > 11].index)

In [19]:
# sns.pairplot(df[['carat', 'depth', 'table', 'x','y','z','price']], diag_kind='kde')

In [20]:
# plt.scatter(df['x'], df['y'])
# xx1 = np.array([3, 11])
# yy1 = np.array([3+0.5, 11+0.5])
# xx2 = np.array([3, 11])
# yy2 = np.array([3-0.6, 11-0.6])
# plt.plot(xx1, yy1)
# plt.plot(xx2, yy2)
# plt.show()

In [21]:
df = df.drop(df[df['y'] > df['x'] + 0.5].index)
df = df.drop(df[df['y'] < df['x'] - 0.61].index)

In [22]:
# plt.scatter(df['x'], df['y'])
# xx1 = np.array([3, 11])
# yy1 = np.array([3+0.5, 11+0.5])
# xx2 = np.array([3, 11])
# yy2 = np.array([3-0.6, 11-0.6])
# plt.plot(xx1, yy1)
# plt.plot(xx2, yy2)
# plt.show()

In [23]:
# plt.scatter(df['x'], df['z'])
# xx1 = np.array([3, 11])
# yy1 = np.array([1.5+0.75, 5.5+2.45])
# xx2 = np.array([3, 11])
# yy2 = np.array([1.5-0.2, 5.5+0.5])
# plt.plot(xx1, yy1, color='yellow')
# plt.plot(xx2, yy2, color='red')
# plt.show()

In [24]:
df = df.drop(df[df['z'] > 0.7125 * df['x'] + 0.1125].index)
df = df.drop(df[df['z'] < 0.5875 * df['x'] - 0.4625].index)

In [25]:
# plt.scatter(df['x'], df['z'])
# xx1 = np.array([3, 11])
# yy1 = np.array([0.7125 * 3 + 0.1125, 0.7125 * 11 + 0.1125])
# xx2 = np.array([3, 11])
# yy2 = np.array([0.5875 * 3 - 0.4625, 0.5875 * 11 - 0.4625])
# plt.plot(xx1, yy1, color='yellow')
# plt.plot(xx2, yy2, color='red')
# plt.show()

In [26]:
# plt.scatter(df['y'], df['z'])
# xx1 = np.array([3, 11])
# yy1 = np.array([1.5+0.75, 5.5+2.45])
# xx2 = np.array([3, 11])
# yy2 = np.array([1.5-0.2, 5.5+0.5])
# plt.plot(xx1, yy1, color='yellow')
# plt.plot(xx2, yy2, color='red')
# plt.show()

In [27]:
# sns.pairplot(df[['carat', 'depth', 'table', 'x','y','z','price']], diag_kind='kde')

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

В ходе нашей работы мы рассматривали различные подходы и пробовали различные модели, такие как разные виды линейной регрессии и KNN. 
Линейная регрессия без генерации большого количества полиномиальных фичей выдавала плохие результаты. С генерацией фичей результаты были средними. KNN же сразу с default параметрами выдавал хороший скор. Оставалось сделать нескоько дополнительных шагов, подобрать параметры и сделать prediction.

In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

Здесь мы создаем две фичи, которые сильно прибавляют скор - x * y * z и table * depth.
Также прибавляем по +2 к лейбл фичам, чтобы избавиться от нулей в признаках и делаем логарифмирование (оно хорошо себя показало на этом датасете, скорее всего из-за распределения цены и других признаков).

In [29]:
df_y = df['price']
df['xyz'] = df['x']*df['y']*df['z']
df['t*d'] = df['table'] * df['depth']
df_x = df.drop(['price'], axis=1)
df_x['cut'] += 2
df_x['color'] += 2
df_x['clarity'] += 2
df_x = np.log(df_x)
df_y = np.log(df_y)

Так же скейлим все фичи, чтобы они были в одном диапазоне - так лучше для нашего предиктора. 
Мы протестировали MinMaxScaler, RobustScaler и StandardScaler. Лучшие результаты показал MinMax.

In [30]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc = MinMaxScaler()
df_x = pd.DataFrame(sc.fit_transform(df_x), index=df.index)

Разбиваем выборку на трейн и тест (это нужно было для тестирования и обучения, а так же кроссвалидации и подбора параметров).

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=99)

In [32]:
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

Дальше мы пробовали сокращать признаковое пространство с помощью PCA.
Аккуратно поэксперементировав мы сократили признаки до 5 и это сильно улучшило нашу модель.

In [33]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
x_train = pca.fit_transform(x_train_copy)
x_test = pca.transform(x_test_copy)

In [34]:
df_x = pca.transform(df_x)

Далее проводим все преобразования с выборкой для сабмита, обучаем модель и делаем предикт.

In [35]:
testing['x'][testing['x'] == 0] += 0.00000001
testing['y'][testing['y'] == 0] += 0.00000001
testing['z'][testing['z'] == 0] += 0.00000001

<ipython-input-35-acb4f4a39808>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing['x'][testing['x'] == 0] += 0.00000001
<ipython-input-35-acb4f4a39808>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing['y'][testing['y'] == 0] += 0.00000001
<ipython-input-35-acb4f4a39808>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing['z'][testing['z'] == 0] += 0.00000001


In [36]:
testing['xyz'] = testing['x']*testing['y']*testing['z']
testing['t*d'] = testing['table'] * testing['depth']
testing['cut'] += 2
testing['color'] += 2
testing['clarity'] += 2
testing = np.log(testing)

In [37]:
testing = sc.transform(testing)

In [38]:
testing = pca.transform(testing)

In [39]:
clf2_knn = KNeighborsRegressor(algorithm='kd_tree', weights='distance', p=2, leaf_size=5, n_neighbors=9)
clf2_knn.fit(df_x , df_y)

KNeighborsRegressor(algorithm='kd_tree', leaf_size=5, n_neighbors=9,
                    weights='distance')

In [40]:
submit_final = pd.DataFrame(np.exp(clf2_knn.predict(testing)), columns=['price'])
submit_final.index += 1
ToCSV(submit_final, 'exp_pca_final_sub_1')

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [41]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 5 sec


# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try 3rd order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA and understand non-linear relation in I/O. Are any interactions more special than others (E.g. is diamond *volume* a better predictor?)
1. If you note a shift in trend for any predictors, try building a model for each trend.
1. Evaluate predictions and focus on poorly predicted "groups". Can they be identified and modeled separately?
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support of approximately linear X-Y relation?
1. How are categorical features treated by the SKLearn models? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features? 
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn about [modeling price of a diamond](https://www.google.com/search?q=machine+learning+model+price+diamond).
1. Try post-processing: shifting/scaling/transforming the distribution of predicted prices `pY` to match the distribution of training prices `tY`
